In [1]:
import os
import numpy as np
import torch
from torch import nn, optim
from torchsummary import summary
import torch.nn.functional as F
from pennylane import *
from classical_unet import *
from train import *
from unitary import * 
from subdataset import dataset_images_train
from QCUnet import * 
from QCUBottleNeck import * 
import pandas as pd

w:\summer school\Methane leaks\base research\code\Methane-Plume-Segmentation-main\Our_QCNN\QuantaMinds\src\train.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


cpu
cpu


## Benchmarking QCUnet models

In [4]:
##  executions

archi_num ='Archi1'

def QCModels_Benchmarking(archi_num ='Archi1'): 
    """
        [True, False, False] ==> working (Archi1)
        [True, True, False] ==> working  (Archi2)
        [False, False, True] ==> working (Archi3)
        [True, False, True] ==> working  (Archi4)
        [True, True, True] ===> working => we do just one experiment with the best params of the previous ones
    """
    experimets_results_path = f'results\\df_exepriments_{archi_num}.csv'
    if os.path.exists(experimets_results_path):
        df_experiments = pd.read_csv(experimets_results_path)
    else :
        df_experiments= pd.DataFrame(columns = [
                                                'archi',
                                                'model_name', 
                                                'epochs', 
                                                'batch_size',
                                                'bands', 
                                                'lr', 
                                                'unitary',  
                                                "opt", 
                                                "loss",                                          
                                                "train_loss", 
                                                "val_loss", 
                                                "train_iou",
                                                "train_acc", 
                                                "metrics_by_epoch"
                                                ])
        df_experiments.to_csv(experimets_results_path)
    # fixed 
    image_size = 90
    bs = 10
    ep = 5

    # varying []
    bandss = [ [12,13], [1,2,3,4,5,6,7,8,9,10,11,12,13]] 
    lr = [0.001,0.01 , 0.05]
    lr = [0.001,0.01]
    
    # Unitaries = [None, U_TTN, U_5, U_6, U_9, U_13, U_14, U_15, U_SO4, U_SU4]
    # Unitaries = [None, U_TTN, U_5]
    Unitaries = [None]
    architectures = {'Archi1' : [True, False, False], 
                     'Archi2': [True, True, False], 
                     'Archi3' : [False, False, True], 
                     'Archi4' : [True, False, True],  
                     'Archi5': [True, True, True]   
                    }
    
    for bands in bandss: 
        n_channels = len(bands)
        for unitary in Unitaries:
            quantum_layers= architectures[archi_num]
            if quantum_layers == [False, False, True]:
                model = UNetWithQBottleNeck(n_channels=n_channels, n_classes=1, unitary=unitary)
            else:
                model = QCUNet(n_channels=n_channels, quantum_layers= quantum_layers ,n_classes=1, 
                            unitary=unitary)
                
            for learning_rate in lr:                 
                # initialize loss function
                loss = nn.BCEWithLogitsLoss()

                # initialize optimizer
                opt = optim.Adam(model.parameters(),lr=learning_rate)

                # run training
                trained_model, model_metrics, overall_metrics, model_name = train_model(
                                            model, ep, opt, 
                                            loss, bs, bands= bands, 
                                            path_savemodel = f'./mod/quantum/{archi_num}/',
                                            lr= learning_rate,  
                                            train_size=200, 
                                            test_size=50, 
                                            TrainImgfiles = dataset_images_train
                                            )
                row= {
                        'archi': archi_num, 
                        'model_name': model_name, 
                        'epochs': ep, 
                        'batch_size': bs,
                        'bands': bands, 
                        'lr': learning_rate, 
                        'unitary': unitary, 
                        "opt" : 'Adam', 
                        "loss" : 'BCEWithLogitsLoss', 
                        "train_loss": overall_metrics["train_loss"], 
                        "val_loss": overall_metrics["val_loss"], 
                        "train_iou" : overall_metrics["train_iou"],
                        "train_acc": overall_metrics["train_acc"] , 
                        "metrics_by_epoch": model_metrics
                }
                
                df_experiments.loc[len(df_experiments)] = row 
                df_experiments.to_csv(experimets_results_path, index=False)
                
QCModels_Benchmarking(archi_num = archi_num)

## Benchmarking Classical Unet

In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

# set seed
set_all_seeds(21)

# model
bands = [1,2,3,4,5,6,7,8,9,10,11,12,13]
model = UNet(n_channels=13, n_classes=1)
model.to(device)

# trainning parameters
ep = 5
learning_rate = 0.001
bs = 10

# setup tensorboard writer

# initialize loss function
loss = nn.BCEWithLogitsLoss()

# initialize optimizer
opt = optim.Adam(model.parameters(),lr=learning_rate)

# run training
# trained_model = train_model(model, ep, opt, loss, bs, bands= bands, lr= lr)

experimets_results_path = f'results\\df_exepriments_classical.csv'
if os.path.exists(experimets_results_path):
        df_experiments = pd.read_csv(experimets_results_path)
else :
        df_experiments= pd.DataFrame(columns = [
                                        'archi',
                                        'model_name', 
                                        'epochs', 
                                        'batch_size',
                                        'bands', 
                                        'lr', 
                                        'unitary',  
                                        "opt", 
                                        "loss",                                          
                                        "train_loss", 
                                        "val_loss", 
                                        "train_iou",
                                        "train_acc", 
                                        "metrics_by_epoch"
                                        ])
df_experiments.to_csv(experimets_results_path)

trained_model, model_metrics, overall_metrics, model_name = train_model(
                                            model, ep, opt, 
                                            loss, bs, bands= bands, 
                                            path_savemodel = f'./mod/classical/',
                                            lr= learning_rate,  
                                            train_size=200, 
                                            test_size=50, 
                                            TrainImgfiles = dataset_images_train
                                            )
row= {
        'archi': 'classical', 
        'model_name': model_name, 
        'epochs': ep, 
        'batch_size': bs,
        'bands': bands, 
        'lr': learning_rate, 
        'unitary': None, 
        "opt" : 'Adam', 
        "loss" : 'BCEWithLogitsLoss', 
        "train_loss": overall_metrics["train_loss"], 
        "val_loss": overall_metrics["val_loss"], 
        "train_iou" : overall_metrics["train_iou"],
        "train_acc": overall_metrics["train_acc"] , 
        "metrics_by_epoch": model_metrics
}

df_experiments.loc[len(df_experiments)] = row 
df_experiments.to_csv(experimets_results_path, index=False)



cpu
20_12_17, 21
epoch 0
batch  0
Train/Loss tensor(0.6116, grad_fn=<DivBackward0>)
Train/Iou 0.10547297
Train/Acc tensor(0.7645)
batch  1
Train/Loss tensor(0.5956, grad_fn=<DivBackward0>)
Train/Iou 0.057835244
Train/Acc tensor(0.7492)
batch  2
Train/Loss tensor(0.5810, grad_fn=<DivBackward0>)
Train/Iou 0.044002056
Train/Acc tensor(0.7949)
batch  3
Train/Loss tensor(0.5522, grad_fn=<DivBackward0>)
Train/Iou 0.04003014
Train/Acc tensor(0.8338)
batch  4
Train/Loss tensor(0.5394, grad_fn=<DivBackward0>)
Train/Iou 0.03647445
Train/Acc tensor(0.8507)
batch  5
Train/Loss tensor(0.5206, grad_fn=<DivBackward0>)
Train/Iou 0.03187259
Train/Acc tensor(0.8664)
batch  6
Train/Loss tensor(0.5002, grad_fn=<DivBackward0>)
Train/Iou 0.027319362
Train/Acc tensor(0.8818)
batch  7
Train/Loss tensor(0.4831, grad_fn=<DivBackward0>)
Train/Iou 0.024809832
Train/Acc tensor(0.8932)
batch  8
Train/Loss tensor(0.4705, grad_fn=<DivBackward0>)
Train/Iou 0.022053184
Train/Acc tensor(0.9024)
batch  9
Train/Loss tenso